<a href="https://colab.research.google.com/github/ducleins/eurusd_2026/blob/main/02_Notebook/02_Clean_Sheet1_Dealer_CTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# CELL 1: SETUP VÀ KHAI BÁO BIẾN
# =========================================================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# 1. Kết nối Google Drive (Chỉ chạy nếu Colab chưa kết nối)
# drive.mount('/content/drive')

# 2. Định nghĩa các đường dẫn chính
BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_CFTC_PATH = os.path.join(BASE_PATH, '01_Raw', 'cftc')
FILTERED_PATH = os.path.join(BASE_PATH, '01_Raw', 'filtered')

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(FILTERED_PATH, exist_ok=True)

# 3. Ánh xạ tên cột gốc sang tên chuẩn (Dựa trên 4 file thành công)
COLUMN_MAPPING = {
    'Report_Date_as_YYYY-MM-DD': 'Date',
    'Open_Interest_All': 'Open_Interest_All',
    'Dealer_Positions_Long_All': 'Dealer_Long_All',
    'Dealer_Positions_Short_All': 'Dealer_Short_All',
    'Lev_Money_Positions_Long_All': 'CTA_Long_All',
    'Lev_Money_Positions_Short_All': 'CTA_Short_All',
    'Market_and_Exchange_Names': 'Market_and_Exchange_Names' # Cột quan trọng để lọc
}

In [2]:
# =========================================================
# CELL 2: HÀM XỬ LÝ DỮ LIỆU VÀ TÍNH TOÁN CHỈ SỐ CƠ BẢN
# =========================================================

def process_cftc_file(file_path):
    """
    Chức năng: Đọc file CFTC, lọc theo 'EURO FX', tính Dealer/CTA Net/Pct.
    Xử lý linh hoạt cho file có/không có header.
    """

    # 1. THỬ ĐỌC FILE (Thử với header trước, delimiter là ',')
    try:
        df = pd.read_csv(file_path, sep=',', encoding='utf-8')
        # Kiểm tra sự tồn tại của cột lọc. Nếu không có, có thể header bị lỗi.
        if COLUMN_MAPPING['Market_and_Exchange_Names'] not in df.columns:
            raise ValueError("Lỗi tiêu đề.")
        is_header_present = True

    except (KeyError, ValueError, pd.errors.ParserError):
        # 1b. THỬ ĐỌC LẠI MÀ KHÔNG CÓ HEADER
        df = pd.read_csv(file_path, sep=',', encoding='utf-8', header=None)
        is_header_present = False

        # Gán tên cột theo Index cho các cột cần thiết (Dựa trên vị trí đã kiểm tra)
        COL_INDICES_FOR_FILTER = {
            0: 'Market_and_Exchange_Names', # Cột 0: Tên Hàng Hóa/Tiền Tệ
        }
        df.rename(columns=COL_INDICES_FOR_FILTER, inplace=True)

    # 2. LỌC DỮ LIỆU CỦA EURUSD
    # Lọc theo mã chuẩn 'EURO FX'
    df_euro = df[df['Market_and_Exchange_Names'].str.contains('EURO FX', na=False, case=False)].copy()

    if df_euro.empty:
        raise ValueError("Không tìm thấy dữ liệu 'EURO FX' sau khi lọc.")

    # 3. CHUẨN HÓA CỘT VÀ ÁNH XẠ

    # Nếu không có header, phải gán tên cột cho các chỉ số vị thế theo Index
    if not is_header_present:
        COL_INDICES_FULL = {
            # 0 đã gán, 2 là Date
            2: 'Report_Date_as_YYYY-MM-DD',
            7: 'Open_Interest_All',
            8: 'Dealer_Positions_Long_All',
            9: 'Dealer_Positions_Short_All',
            14: 'Lev_Money_Positions_Long_All',
            15: 'Lev_Money_Positions_Short_All',
        }
        df_euro.rename(columns=COL_INDICES_FULL, inplace=True)

    # Lấy các cột cần thiết và đổi tên
    df_clean = df_euro[list(COLUMN_MAPPING.keys())].rename(columns=COLUMN_MAPPING)

    # 4. TÍNH TOÁN CHỈ SỐ CƠ BẢN
    df_clean['Date'] = pd.to_datetime(df_clean['Date'])

    # Dealer Net
    df_clean['Dealer_Net'] = df_clean['Dealer_Long_All'] - df_clean['Dealer_Short_All']
    df_clean['Dealer_%OI'] = (df_clean['Dealer_Net'] / df_clean['Open_Interest_All']) * 100

    # CTA Net
    df_clean['CTA_Net'] = df_clean['CTA_Long_All'] - df_clean['CTA_Short_All']
    df_clean['CTA_%OI'] = (df_clean['CTA_Net'] / df_clean['Open_Interest_All']) * 100

    # Giữ lại các cột cần xuất
    FINAL_COLS = ['Date', 'Open_Interest_All', 'Dealer_Net', 'Dealer_%OI', 'CTA_Net', 'CTA_%OI']

    return df_clean[FINAL_COLS]


# =========================================================
# CELL 3: THỰC THI GỘP TẤT CẢ FILE VÀ LƯU
# =========================================================

def combine_and_save_cftc():
    """Chức năng: Quét, xử lý, gộp tất cả file CFTC và lưu ra file filtered."""

    FILE_NAMES = [f for f in os.listdir(RAW_CFTC_PATH) if f.endswith('.txt') or f.endswith('.csv')]
    all_data = []

    print(f"--- BẮT ĐẦU XỬ LÝ {len(FILE_NAMES)} FILE CFTC RAW ---")

    for filename in FILE_NAMES:
        file_path = os.path.join(RAW_CFTC_PATH, filename)

        try:
            df_processed = process_cftc_file(file_path)
            all_data.append(df_processed)
            print(f"✅ Xử lý thành công: {filename}")

        except Exception as e:
            # Nếu có lỗi, chỉ in ra cảnh báo và bỏ qua file lỗi đó
            print(f"⚠️ BỎ QUA file lỗi {filename}: {e}")
            continue

    # Gộp dữ liệu và làm sạch
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df = combined_df.sort_values('Date').drop_duplicates(subset=['Date'])

        # Lưu ra file cuối cùng
        output_file = os.path.join(FILTERED_PATH, 'cftc_dealer_cta_filtered.csv')
        combined_df.to_csv(output_file, index=False)

        print(f"\n--- HOÀN THÀNH XỬ LÝ CFTC ---")
        print(f"Lưu file tại: {output_file}")
        print(f"Kích thước dữ liệu cuối cùng: {combined_df.shape}")

    else:
        print("\n❌ LỖI NGHIÊM TRỌNG: Không có file nào được xử lý thành công.")

# Chạy hàm chính
combine_and_save_cftc()

--- BẮT ĐẦU XỬ LÝ 5 FILE CFTC RAW ---
✅ Xử lý thành công: FinFut_2022.txt
✅ Xử lý thành công: FinFut_2023.txt
✅ Xử lý thành công: FinFut_2024.txt
✅ Xử lý thành công: FinFut_2025.txt
✅ Xử lý thành công: FinFut_21102025.txt

--- HOÀN THÀNH XỬ LÝ CFTC ---
Lưu file tại: /content/drive/MyDrive/EURUSD_2026/01_Raw/filtered/cftc_dealer_cta_filtered.csv
Kích thước dữ liệu cuối cùng: (199, 6)


In [6]:
# =========================================================
# CELL 3: SETUP & MASTER JOIN (TỐI GIẢN CHO SHEET 1)
# =========================================================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- A. MOUNT DRIVE ---
try:
    drive.mount('/content/drive')
except ValueError:
    print("Drive đã được Mount. Bỏ qua Mount.")

# 1. Định nghĩa các đường dẫn (Đảm bảo đã chạy CELL 1)
BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_PATH = os.path.join(BASE_PATH, '01_Raw')
FILTERED_PATH = os.path.join(RAW_PATH, 'filtered')
CLEAN_PATH = os.path.join(BASE_PATH, '02_Clean')
os.makedirs(CLEAN_PATH, exist_ok=True)

# --- B. ĐỌC DỮ LIỆU CFTC ĐÃ LỌC (Cột chính) ---
CFTC_FILTERED_FILE = os.path.join(FILTERED_PATH, 'cftc_dealer_cta_filtered.csv')
df_master = pd.read_csv(CFTC_FILTERED_FILE, parse_dates=['Date']).set_index('Date', drop=True)
print("✅ Đã đọc dữ liệu CFTC đã lọc.")


# --- C. ĐỌC DỮ LIỆU FRED (CHỈ GỘP DGS10 cho TenY_Phase) ---
FRED_DIR = os.path.join(RAW_PATH, 'fred')
# CHỈ LẤY DGS10.csv
fred_files = [f for f in os.listdir(FRED_DIR) if f == 'FRED_DGS10.csv']

for filename in fred_files:
    file_path = os.path.join(FRED_DIR, filename)
    col_name = filename.replace('FRED_', '').replace('.csv', '')

    df_fred = pd.read_csv(
        file_path,
        index_col=0,
        parse_dates=True,
        header=0
    )
    df_fred.columns = [col_name]
    df_master = df_master.join(df_fred, how='outer')

print(f"✅ Đã gộp FRED DGS10 (cần thiết cho TenY_Phase).")


# --- D. ĐỌC DỮ LIỆU YFINANCE ĐÃ CHUẨN HÓA ---
YFINANCE_CLEAN_FILE = os.path.join(FILTERED_PATH, 'yfinance_EURUSD_clean.csv')

# Đọc file đã được chuẩn hóa (chỉ có cột Date và EURUSD_Close)
df_yf_clean = pd.read_csv(
    YFINANCE_CLEAN_FILE,
    index_col=0,            # Cột 0 là cột Date
    parse_dates=True,
    header=0
)
# Cột đã được đổi tên thành EURUSD_Close
df_euraud_close = df_yf_clean['EURUSD_Close']
df_master = df_master.join(df_euraud_close, how='outer')

print("✅ Đã gộp tỷ giá EURUSD đã chuẩn hóa (chỉ cột Close).")

# --- E. LÀM SẠCH VÀ CHUẨN HÓA MASTER DF (SỬA LỖI TYPEERROR) ---
df_master.index = pd.to_datetime(df_master.index) # ÉP KIỂU index về datetime (Khắc phục TypeError)
df_master = df_master.sort_index()
df_master = df_master.ffill().bfill()

print(f"\n--- TỔNG QUAN MASTER DATAFRAME ---")
print(f"Kích thước: {df_master.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã đọc dữ liệu CFTC đã lọc.
✅ Đã gộp FRED DGS10 (cần thiết cho TenY_Phase).
✅ Đã gộp tỷ giá EURUSD đã chuẩn hóa (chỉ cột Close).

--- TỔNG QUAN MASTER DATAFRAME ---
Kích thước: (2069, 7)


In [7]:
# =========================================================
# CELL 4: TÍNH TOÁN CÁC CHỈ SỐ SHEET 1 (ĐÃ BAO GỒM PHASE)
# =========================================================

# Định nghĩa các cửa sổ (chuẩn hóa theo tuần)
WINDOW_3Y = 156
WINDOW_1Y = 52
WINDOW_1M = 4

# --- A. TÍNH TOÁN CÁC CHỈ SỐ CFTC (SHEET 1) ---

# 1. Dealer %OI và Z3Y_Dealer
df_master['Dealer_Net_OI'] = df_master['Dealer_Net'] / df_master['Open_Interest_All'] * 100
df_master['Dealer_%OI'] = df_master['Dealer_Net_OI'] # Đổi tên cho nhất quán với yêu cầu
df_master['Z3Y_Dealer'] = (
    df_master['Dealer_%OI'] - df_master['Dealer_%OI'].rolling(WINDOW_3Y).mean()
) / df_master['Dealer_%OI'].rolling(WINDOW_3Y).std()

# 2. CTA %OI và Z3Y_CTA
df_master['CTA_Net_OI'] = df_master['CTA_Net'] / df_master['Open_Interest_All'] * 100
df_master['CTA_%OI'] = df_master['CTA_Net_OI'] # Đổi tên cho nhất quán với yêu cầu
df_master['Z3Y_CTA'] = (
    df_master['CTA_%OI'] - df_master['CTA_%OI'].rolling(WINDOW_3Y).mean()
) / df_master['CTA_%OI'].rolling(WINDOW_3Y).std()


# --- B. TÍNH TOÁN CÁC CHỈ SỐ PHASE (NHƯ YÊU CẦU CỦA BẠN) ---

# Định nghĩa ngưỡng cho Phase (Dùng cho cả Dealer và CTA)
CONDITIONS = [
    (df_master['Z3Y_Dealer'] <= -1.8),
    (df_master['Z3Y_Dealer'] > -1.8) & (df_master['Z3Y_Dealer'] <= -0.8),
    (df_master['Z3Y_Dealer'] > -0.8) & (df_master['Z3Y_Dealer'] <= 1.5),
    (df_master['Z3Y_Dealer'] > 1.5) & (df_master['Z3Y_Dealer'] <= 1.8),
    (df_master['Z3Y_Dealer'] > 1.8)
]
PHASE_VALUES = ['Rebuild', 'Accumulate', 'Crowded', 'Unwind', 'Rebuild'] # Tùy chỉnh Unwind -> Rebuild (Long Max) theo logic tài liệu

# Dealer_Phase
df_master['Dealer_Phase'] = np.select(CONDITIONS, PHASE_VALUES, default='Neutral')

# CTA_Phase (Áp dụng logic tương tự cho CTA)
CONDITIONS_CTA = [c.replace('Z3Y_Dealer', 'Z3Y_CTA') for c in CONDITIONS] # Tái sử dụng logic so sánh
df_master['CTA_Phase'] = np.select([
    (df_master['Z3Y_CTA'] <= -1.8),
    (df_master['Z3Y_CTA'] > -1.8) & (df_master['Z3Y_CTA'] <= -0.8),
    (df_master['Z3Y_CTA'] > -0.8) & (df_master['Z3Y_CTA'] <= 1.5),
    (df_master['Z3Y_CTA'] > 1.5) & (df_master['Z3Y_CTA'] <= 1.8),
    (df_master['Z3Y_CTA'] > 1.8)
], PHASE_VALUES, default='Neutral')


# --- C. TÍNH CÁC CHỈ SỐ KHÁC ---

# 3. Dealer Imbalance ROC (4 tuần)
df_master['Dealer_Net_Shift4'] = df_master['Dealer_Net'].shift(WINDOW_1M)
df_master['Dealer_Imbalance_ROC'] = np.where(
    df_master['Dealer_Net_Shift4'].abs() != 0,
    (df_master['Dealer_Net'] - df_master['Dealer_Net_Shift4']) / df_master['Dealer_Net_Shift4'].abs() * 100,
    0
)

# 4. CTA Speed
df_master['CTA_Net_Shift4'] = df_master['CTA_Net'].shift(WINDOW_1M)
df_master['CTA_Speed_Denominator'] = df_master['CTA_Net_Shift4'].abs().rolling(WINDOW_1Y).mean()
df_master['CTA_Speed'] = np.where(
    df_master['CTA_Speed_Denominator'] != 0,
    (df_master['CTA_Net'] - df_master['CTA_Net_Shift4']) / df_master['CTA_Speed_Denominator'],
    0
)

# 5. CTA vs Price Divergence (Correlation 52 tuần)
df_master['EURUSD_Pct_Change_4W'] = df_master['EURUSD_Close'].pct_change(WINDOW_1M)
df_master['CTA_Pct_Change_4W'] = df_master['CTA_Net'].pct_change(WINDOW_1M)
df_master['Corr_CTA_Price'] = df_master['CTA_Pct_Change_4W'].rolling(WINDOW_1Y).corr(df_master['EURUSD_Pct_Change_4W'])
df_master['CTA_Price_Divergence'] = df_master['Corr_CTA_Price'].clip(-5, 5)

# 6. TenY_Phase
df_master['DGS10_Mean_156'] = df_master['DGS10'].rolling(WINDOW_3Y).mean()
df_master['DGS10_Std_156'] = df_master['DGS10'].rolling(WINDOW_3Y).std()
df_master['TenY_Phase'] = (df_master['DGS10'] - df_master['DGS10_Mean_156']) / df_master['DGS10_Std_156']

print("✅ Đã tính toán tất cả 10 chỉ số cần thiết cho Sheet 1.")

✅ Đã tính toán tất cả 10 chỉ số cần thiết cho Sheet 1.


In [8]:
# =========================================================
# CELL 5: LỌC VÀ XUẤT DỮ LIỆU SẠCH (SHEET 1)
# =========================================================

# Các chỉ số cần có của Sheet 1 (Tất cả 10 chỉ số bạn liệt kê)
SHEET1_COLS = [
    'Dealer_%OI',
    'Z3Y_Dealer',
    'Dealer_Phase',           # Đã được tính toán trong CELL 4
    'Dealer_Imbalance_ROC',
    'CTA_%OI',
    'Z3Y_CTA',
    'CTA_Phase',              # Đã được tính toán trong CELL 4
    'CTA_Speed',
    'CTA_Price_Divergence',
    'TenY_Phase'
]

# Lọc DataFrame chỉ lấy các cột Sheet 1
df_dealer_cta_clean = df_master[SHEET1_COLS].copy()

# Loại bỏ các hàng có ít nhất một giá trị NaN
df_dealer_cta_clean = df_dealer_cta_clean.dropna(how='any').reset_index()

# Xuất ra file clean cho Sheet 1
OUTPUT_FILE_DEALER_CTA = os.path.join(CLEAN_PATH, 'dealer_cta_clean.csv')
df_dealer_cta_clean.to_csv(OUTPUT_FILE_DEALER_CTA, index=False)

print(f"\n✅ Xuất file dealer_cta_clean.csv thành công ({df_dealer_cta_clean.shape[0]} dòng).")
print(f"File được lưu tại: {OUTPUT_FILE_DEALER_CTA}")


✅ Xuất file dealer_cta_clean.csv thành công (1018 dòng).
File được lưu tại: /content/drive/MyDrive/EURUSD_2026/02_Clean/dealer_cta_clean.csv
